#  **Modelo Preditivo - Vencedor de Partidas(%)**
## **Introdução**
Este notebook tem como objetivo desenvolver um modelo preditivo para determinar o time vencedor em uma partida de futebol, utilizando dados fornecidos pela IBM. A análise inclui a preparação dos dados, a construção e a avaliação do modelo, e a visualização dos resultados. O modelo é treinado para prever a probabilidade de vitória do time da casa, do time visitante e de empate.
## **Dados**
Os dados utilizados neste projeto foram fornecidos pela IBM e consistem em informações sobre partidas de futebol e estatísticas dos times. As principais tabelas são:
- **matches.csv**: Contém detalhes sobre as partidas.
- **teams.csv**: Contém estatísticas e informações sobre os times.
## **1. Importação de Bibliotecas e Dados**
O código começa importando as bibliotecas necessárias e carregando os dados dos arquivos CSV.

- **pandas**: Manipulação e análise de dados.
- **numpy**: Cálculos numéricos e operações matemáticas.
- **matplotlib e seaborn**: Criação de gráficos para visualização de dados.
- **sklearn**: Conjunto de ferramentas de machine learning, como divisão de dados, criação de modelos, e avaliação de desempenho.
- **imblearn**: Para técnicas de balanceamento de dados.

In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, auc, roc_curve, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, balanced_accuracy_score

matches = pd.read_csv('matches.csv', sep=';')
teams = pd.read_csv('teams.csv', sep=',')

 Após carregar os dados, filtramos as partidas que já foram completadas, garantindo que estamos utilizando dados de jogos finalizados para o treinamento do modelo.


In [ ]:
matches = matches.query("status == 'complete'")

## **2. Processamento dos Dados**
Os dados são processados para combinar informações das partidas com estatísticas dos times, criando um DataFrame que inclui as características dos times da casa e visitante.

In [160]:
data = []
count = 0;
for index, curr_match in matches.iterrows():
  
  home_team_goals = curr_match.home_team_goal_count

  confrontation = {"winner": 0}
  if(curr_match.home_team_goal_count > curr_match.away_team_goal_count):
    confrontation = {"winner": 1}
  elif(curr_match.home_team_goal_count < curr_match.away_team_goal_count):
    confrontation = {"winner": 2}

  away_data = teams.query('common_name == "{}"'.format(curr_match.away_team_name)).drop(
    columns=['team_name', 'season',	'country'])
  
  home_data = teams.query('common_name == "{}"'.format(curr_match.home_team_name)).drop(
    columns=['team_name', 'season',	'country'])

  away_data = away_data.rename(columns=lambda col: f'{col}_a')
  home_data = home_data.rename(columns=lambda col: f'{col}_h')

  confrontation.update(home_data.iloc[0].to_dict())
  confrontation.update(away_data.iloc[0].to_dict())
  
  data.append(confrontation)


data = pd.DataFrame(data)
data.to_csv('./teams_confrontations.csv', index=False)

A combinação de dados é feita através de consultas às estatísticas dos times no arquivo `teams.csv`, combinando as colunas para que tenhamos os dados do time da casa e do time visitante no mesmo registro. Renomeamos as colunas para distinguir as estatísticas de cada time, utilizando os sufixos `_h` para o time da casa (home) e `_a` para o time visitante (away).


## **3. Filtragem e Seleção de Colunas**
 
Para garantir que o modelo seja treinado com dados consistentes, filtramos o DataFrame para incluir apenas jogos onde ambos os times jogaram o mesmo número de partidas, tanto em casa quanto fora. Isso é feito para eliminar potenciais vieses que poderiam surgir de uma discrepância no número de partidas jogadas por cada time.
 
Após essa filtragem, selecionamos as colunas mais relevantes para o treinamento do modelo. Essas colunas incluem:
- Número de jogos, vitórias, derrotas e empates.
- Gols marcados e sofridos.
- Posição na liga.
- Cartões, faltas e posse de bola.

In [ ]:
max_games_played_h = data['matches_played_h'].max()
max_games_played_a = data['matches_played_a'].max()

max_games_played = min(max_games_played_h, max_games_played_a)

filtered_data = data[(data['matches_played_h'] == max_games_played) & (data['matches_played_a'] == max_games_played)]

matches_played_h_unique = filtered_data['matches_played_h'].nunique()
matches_played_a_unique = filtered_data['matches_played_a'].nunique()

print(f"Número único de partidas jogadas em casa: {matches_played_h_unique}")
print(f"Número único de partidas jogadas fora: {matches_played_a_unique}")

if matches_played_h_unique == 1 and matches_played_a_unique == 1:
    print("Todos os times jogaram o mesmo número de partidas, tanto em casa quanto fora.")
else:
    print("Existem diferentes números de partidas jogadas entre os times.")

In [ ]:
filtered_data.head()

In [163]:
selected_tables = [
    'matches_played_a', 'matches_played_h',
    'wins_h', 'wins_a',
    'draws_h', 'draws_a',
    'losses_h', 'losses_a',
    'points_per_game_h', 'points_per_game_a',
    'league_position_h', 'league_position_a',
    'goals_scored_h', 'goals_scored_a',
    'goals_conceded_h', 'goals_conceded_a',
    'minutes_per_goal_scored_h', 'minutes_per_goal_scored_a',
    'btts_count_h', 'btts_count_a',
    'shots_h', 'shots_a',
    'shots_on_target_h', 'shots_on_target_a',
    'shots_off_target_h', 'shots_off_target_a',
    'first_team_to_score_percentage_h', 'first_team_to_score_percentage_a',
    'fouls_h', 'fouls_a',
    'cards_per_match_h', 'cards_per_match_a',
    'clean_sheets_h', 'clean_sheets_a',
    'goals_scored_home_h', 'goals_scored_away_h',
    'goals_conceded_home_h', 'goals_conceded_away_h',
    'goals_conceded_home_a', 'goals_conceded_away_a',
    'goal_difference_h', 'goal_difference_a',
    'home_advantage_percentage_h',
    'goals_scored_home_a', 'goals_scored_away_a',
    'average_possession_home_a', 'average_possession_away_a',
    'average_possession_home_h', 'average_possession_away_h',
    'win_percentage_h', 'win_percentage_a',
    'btts_percentage_h', 'btts_percentage_a',
]

## **4. Detecção e Tratamento de Outliers**
Embora o código para detecção e tratamento de outliers esteja comentado, ele é útil para o processo de análise, especialmente para garantir a qualidade dos dados. O código a seguir foi utilizado para identificar e tratar outliers nos dados:

In [ ]:
'''for col in data.columns:
    # Verifica se a coluna é numérica
    if np.issubdtype(data[col].dtype, np.number):
        mean = data[col].mean()
        std_dev = data[col].std()

        data['z_score'] = (data[col] - mean) / std_dev
        threshold = 2.5
        data['is_outlier'] = np.abs(data['z_score']) > threshold

        plt.figure(figsize=(10, 6))

        # Plotar todos os pontos
        plt.scatter(data.index, data[col], color='blue', label='Valores')

        # Plotar os outliers
        plt.scatter(data[data['is_outlier'].astype(bool)].index,
                    data[data['is_outlier'].astype(bool)][col],
                    color='red', label='Outliers', marker='o', edgecolor='black')

        # Adicionar títulos e legendas
        plt.title(col)
        plt.xlabel('Índice')
        plt.ylabel('Valores')
        plt.axhline(y=mean, color='gray', linestyle='--', label='Média')
        plt.axhline(y=mean + threshold * std_dev, color='orange', linestyle='--', label='Limite Superior (Média + 3*Desvio Padrão)')
        plt.axhline(y=mean - threshold * std_dev, color='orange', linestyle='--', label='Limite Inferior (Média - 3*Desvio Padrão)')
        # plt.legend()
        plt.grid(True)

        # Mostrar o gráfico
        plt.show()
        sleep(2)
        clear_output(wait=True)  # Clears the output after each iteration

        # Substitui pela média:
        data.loc[data['is_outlier'], col] = mean

        if std_dev == 0:
            data = data.drop(columns=[col])


data = data.drop(columns=['z_score', 'is_outlier'])

data.to_csv('times.csv')'''

1. **Identificação de Outliers**: O código verifica se cada coluna é numérica e calcula a média e o desvio padrão dos dados. A partir desses valores, calcula o escore Z e identifica os outliers com base em um limiar definido.

2. **Visualização dos Outliers**: Utiliza gráficos de dispersão para mostrar os valores e os outliers identificados. As linhas de média e limites superior e inferior são plotadas para melhor visualização.

3. **Tratamento dos Outliers**: Os valores identificados como outliers são substituídos pela média da coluna correspondente. Se o desvio padrão for zero, a coluna é removida, pois não contribui para a análise.

4. **Salvar os Dados**: Após o tratamento dos outliers, os dados limpos são salvos em um arquivo CSV chamado `times.csv`.


A execução repetida desse código possa ser demorada e gerar muitas tabelas, ele é fundamental para garantir que os dados utilizados para treinar o modelo estejam livres de valores extremos que possam distorcer os resultados. O tratamento de outliers melhora a qualidade dos dados e a precisão do modelo preditivo.


## **5. Treinamento e Avaliação do Modelo**

Foi realizada a divisão dos dados em conjuntos de treino e teste. Utilizamos o **RandomForestClassifier**, um algoritmo capaz de lidar com conjuntos de dados complexos e realizar previsões precisas mesmo em cenários onde há muitos recursos disponíveis.

In [ ]:
x = filtered_data[selected_tables]
y = filtered_data['winner']

# Verificar se 'x' e 'y' não estão vazios antes da divisão
print(f"Tamanho de x: {x.shape}")
print(f"Tamanho de y: {y.shape}")

# Se 'x' ou 'y' estiver vazio, exibir uma mensagem e parar o processamento
if x.shape[0] == 0 or y.shape[0] == 0:
    print("O conjunto de dados está vazio após o pré-processamento. Verifique as etapas de filtragem.")
else:
    # Dividir em treino e teste
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    print("Conjunto de dados dividido com sucesso em treino e teste.")

# Dividir em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Treinamos o modelo e avaliamos a sua acurácia, que é uma métrica importante para medir o desempenho inicial do modelo. Em seguida, geramos um relatório de classificação para obter mais detalhes sobre o desempenho nas diferentes classes de resultado (vitória do time da casa, visitante e empate).


In [ ]:
randomForest = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
randomForest.fit(x_train, y_train)

y_pred = randomForest.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred, labels=[1, 2, 0])
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Vitória do time da casa', 'Vitória do time de fora', 'Empate'], yticklabels=['Vitória do time da casa', 'Vitória do time de fora', 'Empate'])
plt.xlabel('Previsão')
plt.ylabel('Atual')
plt.title('Matriz de Confusão')
plt.show()

A partir da análise da matriz de confusão e dos gráficos gerados, conseguimos identificar onde o modelo tem um bom desempenho e onde ele ainda pode ser otimizado. Utilizamos técnicas de balanceamento de dados, como o oversampling, para garantir que o modelo seja treinado com um conjunto de dados balanceado e, assim, melhorar sua performance geral.


## **6. Análise do Balanceamento dos Dados**  ##
Para melhorar a performance do modelo, analisamos o balanceamento do modelo para entender como a distribuição dos dados está afetando no desempenho do modelo.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

data['winner'].value_counts().plot(kind='bar', ax=ax[0])
data['winner'].value_counts().plot(kind='pie', ax=ax[1])

plt.show()

No gráfico acima, pode-se observar um claro desbalanceamento dos valores da coluna ``winner``, com uma predominância de **vitórias da casa**.

## **7. Balanceamento dos Dados**
Para melhorar a performance do modelo, aplicamos técnicas de balanceamento para lidar com a desproporção das classes e resultar em uma base de dados balanceada. 
Essa abordagem evita viés no modelo, além de fazer com que ele seja mais preciso e generalize melhor. Isso é especialmente importante no contexto do projeto, onde a variedade de resultados (vitórias da casa, empates e vitórias do visitante) precisa ser capturada corretamente para que o modelo possa fazer previsões mais confiáveis.

In [ ]:
empate = data.query('winner == 0')
data_cleaned = data.query('winner != 0')

data_cleaned = pd.concat([data_cleaned, empate.sample(n=57, random_state=42)])
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

data_cleaned['winner'].value_counts().plot(kind='bar', ax=ax[0])
data_cleaned['winner'].value_counts().plot(kind='pie', ax=ax[1])

plt.show()

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversample.fit_resample(data_cleaned.drop('winner', axis=1), data_cleaned['winner'])
data_cleaned = pd.concat([X_resampled, y_resampled], axis=1)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

data_cleaned['winner'].value_counts().plot(kind='bar', ax=ax[0])
data_cleaned['winner'].value_counts().plot(kind='pie', ax=ax[1])

plt.show()

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversample.fit_resample(data_cleaned.drop('winner', axis=1), data_cleaned['winner'])

data_cleaned_balanced = pd.concat([X_resampled, y_resampled], axis=1)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))
data_cleaned_balanced['winner'].value_counts().plot(kind='bar', ax=ax[0])
data_cleaned_balanced['winner'].value_counts().plot(kind='pie', ax=ax[1])
plt.show()

## **8. Avaliação do Modelo com Dados Balanceados**
Treinamos e avaliamos o modelo com os dados balanceados para verificar melhorias na performance.

In [ ]:
X_balanced = data_cleaned_balanced[selected_tables]
y_balanced = data_cleaned_balanced['winner']

X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test_balanced = scaler.transform(X_test_balanced)

rf_model_balanced = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model_balanced.fit(X_train_balanced, y_train_balanced)

y_pred_balanced = rf_model_balanced.predict(X_test_balanced)


print(f"Acurácia: {accuracy_score(y_test_balanced, y_pred_balanced) * 100:.2f}%")
print(classification_report(y_test_balanced, y_pred_balanced))


conf_matrix = confusion_matrix(y_test_balanced, y_pred_balanced, labels=[1, 2, 0])
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Vitória do time da casa', 'Vitória do time de fora', 'Empate'], yticklabels=['Vitória do time da casa', 'Vitória do time de fora', 'Empate'])
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de confusão')
plt.show()

## **9. Previsão de Resultados**
Utilizamos o modelo treinado para prever o resultado de uma partida específica.

In [ ]:
def predict_match(home_team, away_team, rf_model_balanced, teams, scaler):
    
    home_data = teams.query('common_name == "{}"'.format(home_team)).drop(columns=['common_name']).iloc[0].to_dict()
    away_data = teams.query('common_name == "{}"'.format(away_team)).drop(columns=['common_name']).iloc[0].to_dict()
    
    home_data = {f'{k}_h': v for k, v in home_data.items()}
    away_data = {f'{k}_a': v for k, v in away_data.items()}
    
    match_data = {**home_data, **away_data}
    
    match_df = pd.DataFrame([match_data])

    missing_cols = [col for col in X_balanced.columns if col not in match_df.columns]
    for col in missing_cols:
        match_df[col] = 0  
  
    match_df = match_df[X_balanced.columns]

    match_df = scaler.transform(match_df)

    prediction = rf_model_balanced.predict_proba(match_df)
    
    return {
        "home_win_prob": prediction[0][1],
        "draw_prob": prediction[0][0],
        "away_win_prob": prediction[0][2]
    }

home_team = "Corinthians"
away_team = "Atlético GO"

prediction = predict_match(home_team, away_team, rf_model_balanced, teams, scaler)
print(f"Probabilidade de vitória do time da casa ({home_team}): {prediction['home_win_prob'] * 100:.2f}%")
print(f"Probabilidade de empate: {prediction['draw_prob'] * 100:.2f}%")
print(f"Probabilidade de vitória do time visitante ({away_team}): {prediction['away_win_prob'] * 100:.2f}%")

## **10. Curva ROC**
Plotamos a curva ROC para avaliar a capacidade do modelo de classificar corretamente as partidas.

In [ ]:

y_pred_proba = rf_model_balanced.predict_proba(X_test_balanced)[:, 1] 
y_true = y_test_balanced

fpr, tpr, _ = roc_curve(y_true, y_pred_balanced, pos_label=2)

roc_auc = auc(fpr, tpr)


plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## **11. Balanceamento dos Dados com RandomOverSampler**
- O balanceamento dos dados é feito utilizando a técnica de **Random Over-Sampling** com a classe `RandomOverSampler` da biblioteca `imbalanced-learn`. Essa técnica aumenta a quantidade de exemplos da classe minoritária duplicando-os até igualar a quantidade de exemplos das classes majoritárias, o que ajuda a evitar que o modelo favoreça a classe com mais exemplos durante o treinamento.
- Após aplicar o `RandomOverSampler`, os dados são divididos em conjuntos de treino e teste utilizando a função `train_test_split`, garantindo que 70% dos dados sejam usados para treinar o modelo e 30% para teste.
```python

In [174]:
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(X_balanced, y_balanced)
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.3, random_state=42)

## **12. Escalonamento das Features**
- As features são normalizadas usando o **StandardScaler**, o que transforma os dados para que eles tenham média zero e desvio padrão igual a 1. Isso é importante para melhorar o desempenho de muitos algoritmos de machine learning, especialmente aqueles que utilizam distâncias, como RandomForest.
- O escalonamento é aplicado tanto aos dados de treino quanto aos de teste.

In [175]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

## **13. Definição do Grid de Hiperparâmetros**
- Um dicionário `param_grid` é criado para definir diferentes combinações de hiperparâmetros do modelo **Random Forest** que serão testadas. Os hiperparâmetros incluem:
    - `n_estimators`: Número de árvores na floresta.
    - `max_depth`: Profundidade máxima das árvores.
    - `min_samples_split`: Número mínimo de amostras para dividir um nó.
    - `min_samples_leaf`: Número mínimo de amostras em uma folha.

In [176]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

## **14. Treinamento do Modelo com GridSearchCV**
- O **RandomForestClassifier** é instanciado com a opção `class_weight='balanced'`, para lidar melhor com classes desbalanceadas sem a necessidade de balanceamento adicional.
- Utiliza-se o **GridSearchCV** para buscar as melhores combinações de hiperparâmetros definidos no `param_grid`, com a métrica de avaliação sendo o **F1 macro** e utilizando validação cruzada com 5 folds (cv=5).


In [ ]:
randomForest = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_search = GridSearchCV(randomForest, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(x_train_scaled, y_train)

## **15. Avaliação do Melhor Modelo com Dados Balanceados**
- O melhor modelo encontrado pelo `GridSearchCV` é recuperado com `grid_search.best_estimator_`.
- O modelo é avaliado no conjunto de teste. As métricas de avaliação incluem:
    - **Acurácia**: Percentual de previsões corretas.
    - **F1 Score macro**: Média do F1 score para cada classe, considerando a distribuição de classes desbalanceada.
    - **ROC AUC**: Área sob a curva ROC para a previsão de probabilidades das classes.

In [ ]:
best_model_balanced = grid_search.best_estimator_

y_pred_balanced = best_model_balanced.predict(x_test_scaled)
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
f1_balanced = f1_score(y_test, y_pred_balanced, average='macro')
roc_auc_balanced = roc_auc_score(y_test, best_model_balanced.predict_proba(x_test_scaled), multi_class='ovr')

print(f"Acurácia: {accuracy_balanced * 100:.2f}%")
print(f"F1 Score (macro): {f1_balanced:.2f}")
print(f"AUC-ROC: {roc_auc_balanced:.2f}")
print(classification_report(y_test, y_pred_balanced))

conf_matrix_balanced = confusion_matrix(y_test, y_pred_balanced, labels=[1, 2, 0])
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_balanced, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Vitória Casa', 'Vitória Fora', 'Empate'], 
            yticklabels=['Vitória Casa', 'Vitória Fora', 'Empate'])
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de Confusão com Dados Balanceados')
plt.show()

**Análise das Métricas de Avaliação**
1. **Acurácia**: 
   - A **acurácia** foi de **72.22%**, o que significa que o modelo acertou aproximadamente 72% das previsões. Isso indica um desempenho geral satisfatório, mas existem algumas áreas que precisam de melhorias, como a classificação de vitórias da casa e vitórias do time de fora.

2. **Precision**: A precisão para cada classe reflete a proporção de resultados positivos reais entre aqueles previstos como positivos. A classe **Vitória Fora** (classe 2) tem uma precisão de 0.77, que é um bom valor, enquanto a classe **Empate** (classe 0) tem uma precisão de 0.71.
  
3. **Recall**: O recall mede a proporção de verdadeiros positivos corretamente identificados pelo modelo. A classe **Vitória da Casa** (classe 1) tem o recall mais baixo (0.52), sugerindo que o modelo tem dificuldade em identificar corretamente as vitórias da casa.

4. **F1 Score (macro)**:
   - O **F1 Score macro** é **0.71**, uma média harmônica da precisão e do recall, considerando o equilíbrio entre os diferentes tipos de erro. Este valor sugere que o modelo está balanceado nas previsões entre as classes.
   - A classe **Vitória da Casa** tem um F1 Score menor (0.62), refletindo a dificuldade do modelo em prever corretamente essas vitórias. O F1 Score de **Empate**(0.80) e **Vitória Fora**(0.71) indicam uma performance mais estável para essas classes.

5. **AUC-ROC**:
   - O valor **AUC-ROC** de **0.89** indica uma excelente habilidade de classificação entre as três classes. Este valor próximo de 1 mostra que o modelo é capaz de distinguir bem entre empates, vitórias da casa e vitórias fora.

6. **Matriz de confusão**:
   - **Vitória da Casa (Classe 1)**: O modelo previu corretamente 17 das 33 vitórias da casa. No entanto, houve 9 erros ao prever vitórias da casa como vitórias do time fora, e 7 erros ao prever vitórias como empates.
   - **Vitória do Time de Fora (Classe 2)**: O modelo acertou 27 das 38 vitórias do time de fora. Os erros restantes incluíram 4 casos de previsão como vitória da casa e 7 como empates.
   - **Empates (Classe 0)**: Dos 37 empates, o modelo previu corretamente 34, errando 1 vez ao prever como vitória da casa e 2 vezes ao prever como vitória do time de fora.

**Considerações Finais**

O modelo de Random Forest balanceado apresentou uma performance razoável, com uma **acurácia de 72.22%** e um **AUC-ROC de 0.89**. No entanto, a classe **Vitória da Casa** foi a que apresentou o maior número de erros, refletido também no F1 Score mais baixo. Esse comportamento pode estar relacionado à variabilidade nos fatores que levam um time a vencer em casa, e à dificuldade do modelo em captar essas variações.

## **16. Função para Prever um Jogo Específico**

Utilizamos a mesma função utilizada anteriormente para realizar as previsões.

- A função `predict_match` permite prever o resultado de um jogo entre dois times específicos (informados por `home_team` e `away_team`).
- A função busca as informações dos times na tabela `teams`, ajusta o formato dos dados para que eles sejam compatíveis com o modelo e, em seguida, aplica o modelo para prever as probabilidades de vitória para o time da casa, empate ou vitória do time visitante.
- As probabilidades são retornadas como um dicionário.


In [ ]:
def predict_match(home_team, away_team, model, teams, scaler):
    
    home_data = teams.query('common_name == "{}"'.format(home_team)).drop(columns=['common_name']).iloc[0].to_dict()
    away_data = teams.query('common_name == "{}"'.format(away_team)).drop(columns=['common_name']).iloc[0].to_dict()
    
    home_data = {f'{k}_h': v for k, v in home_data.items()}
    away_data = {f'{k}_a': v for k, v in away_data.items()}
    
    match_data = {**home_data, **away_data}
    
    match_df = pd.DataFrame([match_data])

    missing_cols = [col for col in x_train.columns if col not in match_df.columns]
    for col in missing_cols:
        match_df[col] = 0  
  
    match_df = match_df[x_train.columns]

    match_df = scaler.transform(match_df)

    prediction = model.predict_proba(match_df)
    
    return {
        "home_win_prob": prediction[0][1],
        "draw_prob": prediction[0][0],
        "away_win_prob": prediction[0][2]
    }

home_team = "Corinthians"
away_team = "Atlético GO"

prediction = predict_match(home_team, away_team, best_model_balanced, teams, scaler)
print(f"Probabilidade de vitória do time da casa ({home_team}): {prediction['home_win_prob'] * 100:.2f}%")
print(f"Probabilidade de empate: {prediction['draw_prob'] * 100:.2f}%")
print(f"Probabilidade de vitória do time visitante ({away_team}): {prediction['away_win_prob'] * 100:.2f}%")

## **17. Importância das Features**
- As features que mais influenciam no modelo são apresentadas no gráfico abaixo. A importância de cada feature foi medida com base na capacidade de melhoria no poder de decisão em cada árvore da floresta.
- Algumas features são mais importantes porque possuem uma alta correlação com o resultado, como por exemplo a performance recente dos times e o histórico de vitórias em casa.
- **Justificativa**: Na próxima sprint, será feita uma reavaliação dessas features com base nos resultados obtidos. Novas features poderão ser adicionadas ou removidas conforme necessário.


In [ ]:
# Importar bibliotecas necessárias
importances = best_model_balanced.feature_importances_
indices = np.argsort(importances)[::-1]  # Ordenar as features por importância

# Exibir as features com maior importância
print("Importância das Features:")
for f in range(x_train.shape[1]):
    print(f"{f + 1}. {x_train.columns[indices[f]]} ({importances[indices[f]]:.4f})")

# Plotar a importância das features
plt.figure(figsize=(12, 8))
plt.title("Importância das Features")
plt.bar(range(x_train.shape[1]), importances[indices], align="center")
plt.xticks(range(x_train.shape[1]), [x_train.columns[i] for i in indices], rotation=90)
plt.xlabel('Features')
plt.ylabel('Importância')
plt.tight_layout()
plt.show()

## **18. Métricas Adicionais: Log Loss e Balanced Accuracy**
- Além das métricas já calculadas (acurácia, F1 Score), outras métricas são fundamentais para uma análise mais precisa do desempenho do modelo.
    - **Log Loss**: Mede a incerteza das previsões probabilísticas. Quanto menor, melhor.
    - **Balanced Accuracy**: Leva em consideração a acurácia balanceada, o que é importante para lidar com dados desbalanceados.

As duas métricas serão calculadas a seguir para avaliar a performance do modelo.

In [ ]:
log_loss_value = log_loss(y_test, best_model_balanced.predict_proba(x_test_scaled))

balanced_accuracy_value = balanced_accuracy_score(y_test, y_pred_balanced)

print(f"Log Loss: {log_loss_value:.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_value:.4f}")

**Análise das Métricas**

- **Log Loss**: O valor de **0.6818** indica que o modelo tem um bom nível de confiança em suas previsões. No entanto, ele ainda tem espaço para melhorar em prever corretamente as probabilidades, especialmente em partidas onde há maior incerteza sobre o resultado.

- **Balanced Accuracy**: A **Balanced Accuracy** de **0.7149** indica que o modelo está equilibrado ao lidar com as classes desbalanceadas (por exemplo, vitórias da casa são mais frequentes). Apesar disso, ainda há margem para otimizar as previsões para as classes menos frequentes (vitória fora e empate).

## **Conclusão Geral**
Desenvolvemos um modelo preditivo utilizando o algoritmo Random Forest para prever os resultados de partidas de futebol na Série A do Campeonato Brasileiro.

As métricas de avaliação do modelo indicaram um desempenho satisfatório, com uma **acurácia de 72.22%** e um **AUC-ROC de 0.89**, evidenciando a capacidade do modelo em distinguir entre os diferentes resultados das partidas. No entanto, notamos que a classe **Vitória da Casa** apresentou desafios, com um recall relativamente baixo, o que sugere a necessidade de uma análise mais aprofundada das características que podem impactar esses resultados.

Com base na análise da importância das features, planejamos realizar ajustes nas variáveis utilizadas no modelo para otimizar sua performance. Para a próxima sprint, focaremos em revisar as características que demonstraram maior influência nos resultados, explorando novos dados e potenciais variáveis que possam enriquecer nosso modelo.
